# How to periodically execute things

In [1]:
import asyncio
import time

import app_state

In [2]:
# override print to allow showing the output in one cell as it was printed in a terminal
import os

real_print = print
lines = []
def print(text):
    global lines
    lines.append(text)

In [3]:
max_calls = 5
call_every = 1 / 2

Trivial, straight forward implementation

In [4]:
for call_index in range(max_calls):
    
    print("Hello")
    
    time.sleep(call_every)

In [5]:
real_print("\n".join(lines))
lines = []

Hello
Hello
Hello
Hello
Hello


Not utterly brilliant!

The problem here is:

- something far more complex than `print` might be called.
- we might not now exactly know how long the execution takes.
- execution time might vary depending on the systems load

A better variant might be this.

In [6]:
for call_index in range(max_calls):
    tic  = time.perf_counter()
    print("Hello")
    toc = time.perf_counter()

    sleep_for = max(0, call_every - (toc - tic))
    time.sleep(sleep_for)

In [7]:
real_print("\n".join(lines))
lines = []

Hello
Hello
Hello
Hello
Hello


Much better!

Now generalize this to allow it to be used as a template, make it interruptable and usable in an asyncio based environment.

In [8]:
async def exit_after(exit_after):
    
    await asyncio.sleep(exit_after)
    app_state.keep_running = False

In [9]:
async def periodically(call_every, callback):
    while app_state.keep_running:
        tic  = time.perf_counter()
        callback()
        toc = time.perf_counter()
    
        sleep_for = max(0, call_every - (toc - tic))
        await asyncio.sleep(sleep_for)

In [12]:
app_state.keep_running = True

_ = await asyncio.gather(
    exit_after(5),
    periodically(1 / 2, lambda : print("Hello")),
)

In [11]:
real_print("\n".join(lines))
lines = []

Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
